In [1]:
import wfdb
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# Load record 100
record = wfdb.rdrecord('100')
annotation = wfdb.rdann('100', 'atr')

# Extract signal data and metadata
signal = record.p_signal
fs = record.fs
channels = record.sig_name
duration = len(signal) / fs

print(f"Sampling frequency: {fs} Hz")
print(f"Channels: {channels}")
print(f"Recording duration: {duration:.2f} seconds")

# Select channel MLII (assumed to be channel 0)
channel_idx = channels.index('MLII') if 'MLII' in channels else 0
ml_ii_signal = signal[:, channel_idx]

# Plot a 10-second segment
start_sec = 0
end_sec = 10
start_idx = int(start_sec * fs)
end_idx = int(end_sec * fs)

segment = ml_ii_signal[start_idx:end_idx]
time = np.arange(start_idx, end_idx) / fs

plt.figure(figsize=(12, 4))
plt.plot(time, segment, label='ECG (MLII)')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude [mV]')
plt.title('10-second ECG segment with annotations')

# Overlay annotations in this segment
ann_symbols = annotation.symbol
ann_samples = annotation.sample

# Filter annotations within the plotted segment
ann_in_segment = [(s, sym) for s, sym in zip(ann_samples, ann_symbols) if start_idx <= s < end_idx]

if ann_in_segment:
    ann_times = [(s / fs) for s, _ in ann_in_segment]
    ann_labels = [sym for _, sym in ann_in_segment]
    plt.scatter(ann_times, ml_ii_signal[[s for s, _ in ann_in_segment]], color='red', label='Annotations')

# Create a legend for annotation symbols
unique_symbols = set(ann_labels)
legend_text = ', '.join(unique_symbols)
plt.legend(title=f"Annotations: {legend_text}")

plt.show()

# Summarize heartbeat types in the segment
heartbeat_counts = Counter(ann_labels)
print("Heartbeat type distribution in 10-second segment:")
for beat_type, count in heartbeat_counts.items():
    print(f"{beat_type}: {count}")


FileNotFoundError: [Errno 2] No such file or directory: '/home/karam/Projects/AI/AIC/code/100.hea'

In [ ]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, iirnotch

# Load the record
record = wfdb.rdrecord('100')
signal = record.p_signal[:, 0]  # MLII channel
fs = record.fs

# Select a 10-second segment
start_idx = 0
end_idx = int(10 * fs)
ecq_segment = signal[start_idx:end_idx]
time = np.arange(start_idx, end_idx) / fs

# a. Baseline Wander Removal (high-pass filter using FFT)
ecfft = np.fft.fft(ecq_segment)
frequencies = np.fft.fftfreq(len(ecfft), d=1/fs)
ecfft[np.abs(frequencies) < 0.5] = 0
baseline_removed = np.fft.ifft(ecfft).real

# b. Power Line Interference Removal (50 Hz notch filter)
notch_freq = 50  # or 60 depending on your country
quality_factor = 30
b_notch, a_notch = iirnotch(notch_freq, quality_factor, fs)
powerline_removed = filtfilt(b_notch, a_notch, baseline_removed)

# c. High-frequency Noise Reduction (low-pass filter <40 Hz)
cutoff = 40  # Hz
b_lp, a_lp = butter(4, cutoff / (0.5 * fs), btype='low')
smoothed_signal = filtfilt(b_lp, a_lp, powerline_removed)

# Plot comparisons
plt.figure(figsize=(15, 10))

plt.subplot(4, 1, 1)
plt.plot(time, ecq_segment)
plt.title('Original ECG Segment')
plt.ylabel('Amplitude [mV]')

plt.subplot(4, 1, 2)
plt.plot(time, baseline_removed)
plt.title('After Baseline Wander Removal')
plt.ylabel('Amplitude [mV]')

plt.subplot(4, 1, 3)
plt.plot(time, powerline_removed)
plt.title('After Power Line Interference Removal')
plt.ylabel('Amplitude [mV]')

plt.subplot(4, 1, 4)
plt.plot(time, smoothed_signal)
plt.title('After Noise Reduction (Smoothing)')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude [mV]')

plt.tight_layout()
plt.show()

# Comments on differences
print("\nObserved Differences:")
print("- After baseline removal: drifting low-frequency trends are gone, signal is centered.")
print("- After notch filter: 50 Hz oscillations are suppressed, cleaner waveform.")
print("- After smoothing: spiky high-frequency noise is reduced, morphology (P-QRS-T waves) becomes clearer.")


In [ ]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, iirnotch, find_peaks

# Load the record
record = wfdb.rdrecord('100')
signal = record.p_signal[:, 0]  # MLII channel
fs = record.fs

# Preprocessing (assumed already done)
start_idx = 0
end_idx = int(10 * fs)
ecq_segment = signal[start_idx:end_idx]
time = np.arange(start_idx, end_idx) / fs

# Apply preprocessing filters
ecfft = np.fft.fft(ecq_segment)
frequencies = np.fft.fftfreq(len(ecfft), d=1/fs)
ecfft[np.abs(frequencies) < 0.5] = 0
baseline_removed = np.fft.ifft(ecfft).real

notch_freq = 50
quality_factor = 30
b_notch, a_notch = iirnotch(notch_freq, quality_factor, fs)
powerline_removed = filtfilt(b_notch, a_notch, baseline_removed)

cutoff = 40
b_lp, a_lp = butter(4, cutoff / (0.5 * fs), btype='low')
cleaned_signal = filtfilt(b_lp, a_lp, powerline_removed)

# R-Peak Detection
peaks, _ = find_peaks(cleaned_signal, distance=0.6*fs, height=np.mean(cleaned_signal) + 0.5*np.std(cleaned_signal))

# Plot ECG with detected R-peaks
plt.figure(figsize=(12, 4))
plt.plot(time, cleaned_signal, label='Cleaned ECG')
plt.plot(time[peaks], cleaned_signal[peaks], 'ro', label='R-peaks')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude [mV]')
plt.title('ECG with Detected R-Peaks')
plt.legend()
plt.show()

# Calculate RR intervals
rr_intervals = np.diff(peaks) / fs

# Visualize RR intervals (line plot)
plt.figure(figsize=(10, 4))
plt.plot(rr_intervals, marker='o')
plt.title('RR Intervals Over Beats')
plt.xlabel('Beat Number')
plt.ylabel('Interval [s]')
plt.show()

# Visualize RR intervals (histogram)
plt.figure(figsize=(8, 4))
plt.hist(rr_intervals, bins=10, edgecolor='black')
plt.title('Histogram of RR Intervals')
plt.xlabel('Interval [s]')
plt.ylabel('Count')
plt.show()

# Compute average heart rate
avg_rr = np.mean(rr_intervals)
heart_rate = 60 / avg_rr
print(f"Average Heart Rate: {heart_rate:.2f} bpm")

# Check for abnormalities
if heart_rate < 60:
    print("Bradycardia detected (slow heart rate).")
elif heart_rate > 100:
    print("Tachycardia detected (fast heart rate).")
else:
    print("Normal heart rate.")

# Heart rate variability
std_rr = np.std(rr_intervals)
print(f"RR Interval Standard Deviation: {std_rr:.4f} s")
if std_rr > 0.1:
    print("Noticeable variability, possible arrhythmia or irregular rhythm.")
else:
    print("Stable rhythm, low variability.")

In [ ]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, iirnotch, find_peaks
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_curve, auc
import seaborn as sns

# Assume previous cleaned_signal, rr_intervals, heart_rate, std_rr are already computed

# Prepare features (example: mean RR, std RR, heart rate)
X = np.array([[np.mean(rr_intervals), np.std(rr_intervals), heart_rate]])
y = np.array([1 if (heart_rate < 60 or heart_rate > 100 or np.std(rr_intervals) > 0.1) else 0])  # 1 = abnormal, 0 = normal

# For demonstration, simulate dataset (since we need multiple samples)
np.random.seed(42)
num_samples = 200
X_sim = np.hstack([
    np.random.normal(np.mean(rr_intervals), 0.05, (num_samples, 1)),
    np.random.normal(np.std(rr_intervals), 0.02, (num_samples, 1)),
    np.random.normal(heart_rate, 5, (num_samples, 1))
])
y_sim = np.random.choice([0, 1], size=num_samples)  # Random labels for demo

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_sim, y_sim, test_size=0.3, random_state=42)

# Train classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)

# Evaluate
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {acc:.2f}")
print(f"Precision: {prec:.2f}")
print(f"Recall: {rec:.2f}")
print(f"F1 Score: {f1:.2f}")

# Plot confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Abnormal'], yticklabels=['Normal', 'Abnormal'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# ROC Curve
y_prob = clf.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend()
plt.show()

# Final observation
if heart_rate < 60:
    print("Final Classification: Bradycardia (abnormal).")
elif heart_rate > 100:
    print("Final Classification: Tachycardia (abnormal).")
elif np.std(rr_intervals) > 0.1:
    print("Final Classification: Possible arrhythmia (abnormal variability).")
else:
    print("Final Classification: Normal rhythm.")
